# Mini-Xception 

In [ ]:
# Core Dependencies (for all models)
import os
import numpy as np
import matplotlib.pyplot as plt

# Deep Learning Dependencies
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

# Custom Dependencies (for Mini-Xception)
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import SeparableConv2D, Add, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense

### Load and Pre-process FER2013 Dataset

In [ ]:
# Define the Paths 
dataset_train = 'FER2013/train'
dataset_test = 'FER2013/test'

In [ ]:
# Define parameters


## Trial #5: Mini-Xception

### Create Data Generators 

In [ ]:
# Define Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.15,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    dataset_train,
    target_size=img_size,
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    dataset_train,
    target_size=img_size,
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    dataset_test,
    target_size=img_size,
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

### Define the Model

In [ ]:
# Define model architecture
def mini_xception(input_shape=(48, 48, 3), num_classes=7):
    input_layer = Input(shape=input_shape)
    
    # Initial Block
    x = Conv2D(8, (3, 3), padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Residual Block x4
    filters = [16, 32, 64, 128]
    for f in filters:
        residual = Conv2D(f, (1, 1), strides=(2, 2), padding='same')(x)
        residual = BatchNormalization()(residual)

        x = SeparableConv2D(f, (3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = SeparableConv2D(f, (3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

        x = Add()([x, residual])

    # Final Block
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    output = Dense(num_classes, activation='softmax')(x)


    return Model(inputs=input_layer, outputs=output)

In [ ]:
# Compile the model 
model_5 = mini_xception()

loss_fn = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05)
model_5.compile(optimizer=Adam(learning_rate=1e-4),
              loss=loss_fn,
              metrics=['accuracy'])

model_5.summary()

### Train the Model 

In [ ]:
# Train the model (Test 5)
initial_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5)
]

history = model_5.fit(
    train_generator,
    validation_data=val_generator,
    epochs=40,
    callbacks=initial_callbacks,
    class_weight=class_weights_dict
)

### Evaluate the Model 

In [ ]:
test_loss, test_accuracy = model_5.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

### Plot the Model on Training Histroy 